# 4. Home Credit Default Risk - Stacking New

In [ ]:
import pandas as pd 
import numpy as np
import random
from six.moves import cPickle as pickle

# plotting libs
from matplotlib import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set() # setting seaborn default for plots

In [ ]:
# figure size in inches
rcParams['figure.figsize'] = 13.5, 10

In [ ]:
import os
data_path="../data/pickles"

In [ ]:
prep_train = pickle.load(open(os.path.join(data_path, "prep_train_EF.pkl"), "rb"))
prep_test = pickle.load(open(os.path.join(data_path, "prep_test_EF.pkl"), "rb"))

In [ ]:
print(prep_train.shape)
print(prep_test.shape)

In [ ]:
X_train_SK_IDS = prep_train.SK_ID_CURR
X_train = prep_train.drop(columns=['TARGET', 'SK_ID_CURR'])
y_train = prep_train.TARGET

X_test_SK_IDS = prep_test.SK_ID_CURR
X_test = prep_test.drop(columns=['SK_ID_CURR'])

# Delete 
del prep_train
del prep_test

### Scale the data

In [ ]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_train)
train_column_names = X_train.columns
X_train = pd.DataFrame(std_scaler.transform(X_train), columns=train_column_names)
X_test = pd.DataFrame(std_scaler.transform(X_test), columns=train_column_names)

--------
## Define L1-Learners

In [ ]:
# Class wrapper for L1-Learners with unique_name property
class L1Learner(object):
  def __init__(self, clf, unique_name):
    self.clf = clf
    self.unique_name = unique_name
    self.predictions_proba = []
    self.predictions = []
    self.predictions_proba_test = []
  def __repr__(self):
    return 'Type:{} | Name: {}'.format(self.clf.__class__.__name__, self.unique_name)

### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf_logreg = L1Learner(LogisticRegression(), unique_name='LogisticRegression')
clf_logreg.clf.set_params(**{
  'C': 1,
  'class_weight': None,
  'n_jobs': -1,
  'random_state': 42
})

### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf_rf = L1Learner(RandomForestClassifier(), unique_name='RandomForestClassifier')
clf_rf.clf.set_params(**{
  'max_leaf_nodes': 70,
  'n_estimators': 150,
  'n_jobs': -1,
  'random_state': 42
})

In [ ]:
clf_rf_2 = L1Learner(RandomForestClassifier(), unique_name='RandomForestClassifier_2')
clf_rf_2.clf.set_params(**{
  'n_estimators': 200,
  'max_features': 0.2,
  'max_depth': 12,
  'min_samples_leaf': 2,
  'n_jobs': -1,
  'random_state': 42
})

### XGBoost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
clf_xgb = L1Learner(XGBClassifier(), unique_name='XGBClassifier')
clf_xgb.clf.set_params(**{
  "n_estimators": 1000,
  "max_depth": 4,
  "min_child_weight": 4,
  "subsample": 0.8,
  "learning_rate": 0.01,
  "colsample_bytree": 0.8,
  "objective": 'binary:logistic',
  'n_jobs': -1,
  "random_state": 42
})

In [ ]:
clf_xgb_2 = L1Learner(XGBClassifier(), unique_name='XGBClassifier_2')
clf_xgb_2.clf.set_params(**{
  'colsample_bytree': 0.7,
  'silent': 1,
  'subsample': 0.7,
  'learning_rate': 0.075,
  'objective': 'binary:logistic',
  'max_depth': 4,
  'min_child_weight': 1,
  'n_jobs': -1,
  'random_state': 42
})

In [ ]:
clf_xgb_3 = L1Learner(XGBClassifier(), unique_name='XGBClassifier_MB')
clf_xgb_3.clf.set_params(**{
  'max_depth': 6,
  'subsample': 0.8,
  'colsample_bytree': 0.632,
  'min_child_weight': 35,
  'objective': 'binary:logistic',
  'silent': 1,
  'n_jobs': -1,
  'random_state': 42
})

### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf_lgm = L1Learner(LGBMClassifier(), unique_name='LightGBM')
clf_lgm.clf.set_params(**{
  'n_estimators': 10000,
  'learning_rate': 0.02,
  'num_leaves': 34,
  'colsample_bytree': 0.9497036,
  'subsample': 0.8715623,
  'max_depth': 8,
  'reg_alpha':0.041545473,
  'reg_lambda':0.0735294,
  'min_split_gain': 0.0222415,
  'min_child_weight': 39.3259775,
  'n_jobs': -1,
  'random_state': 42
})

In [ ]:
clf_lgm_2 = L1Learner(LGBMClassifier(), unique_name='LightGBM_2')
clf_lgm_2.clf.set_params(**{
  'n_estimators':200,
  'learning_rate':0.1,
  'num_leaves':123,
  'colsample_bytree':0.8,
  'subsample':0.9,
  'max_depth':15,
  'reg_alpha':0.1,
  'reg_lambda':0.1,
  'min_split_gain':0.01,
  'min_child_weight':2,
  'n_jobs': -1,
  'random_state': 42
})

### SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
clf_sgd = L1Learner(SGDClassifier(), unique_name='SGDClassifier')
clf_sgd.clf.set_params(**{
  'n_iter': 1, 
  'warm_start': True, 
  'penalty':'l2', 
  'loss': 'log', 
  'learning_rate': 'constant', 
  'eta0': 0.0005, 
  'n_jobs':-1,
  'random_state':42
})

### ExtraTreesClassifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

In [ ]:
clf_etc = L1Learner(ExtraTreesClassifier(), unique_name='ExtraTreesClassifier')
clf_etc.clf.set_params(**{
  'n_estimators': 200,
  'max_features': 0.5,
  'max_depth': 12,
  'min_samples_leaf': 2,
  'n_jobs':-1,
  'random_state': 42
})

### CatBoost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
clf_cat = L1Learner(CatBoostClassifier(), unique_name='CatBoosClassifier')
clf_cat.clf.set_params(**{
  'iterations': 200,
  'learning_rate': 0.5,
  'depth': 3,
  'l2_leaf_reg': 40,
  'bootstrap_type': 'Bernoulli',
  'subsample': 0.7,
  'scale_pos_weight': 5,
  'eval_metric': 'AUC',
  'od_type': 'Iter',
  'allow_writing_files': False,
  #'n_jobs':-1,
  'random_state': 42
})

### Multi Layer Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
clf_mlp = L1Learner(MLPClassifier(), unique_name='MLPClassifier')
clf_mlp.clf.set_params(**{
  'alpha': 0.20842163497959337, 
  'hidden_layer_sizes': (70, 30),
  'random_state': 42
})

----
## Train L1-Learners

### Define L1-Learners

In [ ]:
l1_learners = []
#l1_learners.append(clf_logreg)
#l1_learners.append(clf_rf)
#l1_learners.append(clf_rf_2)
#l1_learners.append(clf_xgb)
#l1_learners.append(clf_xgb_2)
#l1_learners.append(clf_xgb_3)
#l1_learners.append(clf_lgm)
#l1_learners.append(clf_lgm_2)
#l1_learners.append(clf_sgd)
#l1_learners.append(clf_etc)
#l1_learners.append(clf_cat)
#l1_learners.append(clf_mlp)

In [ ]:
import time
from sklearn.metrics import roc_auc_score
from os import listdir

def train_l1_learners(skf, l1_learners, X_tr, y_tr, X_te):
  
  print("Number of Learners to train: {}".format(len(l1_learners)))
  
  for learner in l1_learners:
    
    learner.predictions = np.zeros((X_tr.shape[0], 1))
    learner.predictions_proba = np.zeros((X_tr.shape[0], 1))
    learner.predictions_proba_test = np.zeros((X_te.shape[0], skf.n_splits))
    
    start_time = time.time()
    print('Start training {}'.format(learner))
    
    fold = 0
    for train_idx, test_idx in skf.split(X_tr, y_tr):
      print('Fold #{}'.format(fold))
      # split data for cv
      X_train_cv = X_train.iloc[train_idx, :]
      y_train_cv = y_train.iloc[train_idx]
      X_test_cv  = X_train.iloc[test_idx, :]
      y_test_cv  = y_train.iloc[test_idx]
      
      # train clf
      learner.clf.fit(X_train_cv, y_train_cv)
      
      # make predictions on X_pred
      learner.predictions[test_idx, 0] = learner.clf.predict(X_test_cv)
      learner.predictions_proba[test_idx, 0] = learner.clf.predict_proba(X_test_cv)[:, 1]
      
      # make predictions for X_test
      learner.predictions_proba_test[:, fold] = learner.clf.predict_proba(X_te)[:, 1]
      
      # print score
      score = roc_auc_score(y_test_cv, learner.predictions_proba[test_idx, 0])
      print("Score on X_pred for fold {} (ROC AUC): {}".format(fold, score))
      fold += 1
      
    end_time = time.time()
    print("End (Elapsed time is {} min.)".format((end_time - start_time) / 60))
    print(40*'-')

In [ ]:
from sklearn.model_selection import StratifiedKFold

# define StratifiedKFold
skf = StratifiedKFold(n_splits=5, random_state=42)

train_l1_learners(skf, l1_learners, X_train, y_train, X_test)

### Save/Load models and predictions

In [ ]:
from sklearn.externals import joblib
stacking_models_dir = './stacking_models'

stacking_model_prefix = 'stacking_CV_'
#stacking_model_prefix = 'stacking_60_'
#stacking_model_prefix = 'stacking_75_'

In [ ]:
# load learners
l1_learners = []

for f in os.listdir(stacking_models_dir):
  if f.startswith(stacking_model_prefix):
    l1_learners.append(joblib.load("{}/{}".format(stacking_models_dir, f)))
    
l1_learners

In [ ]:
# save learners
for learner in l1_learners:
  joblib.dump(learner, '{}/{}{}.pkl'.format(stacking_models_dir, stacking_model_prefix, learner.unique_name))

---
## Create new dataset with L1-Learners predictions + extra features

Add these features to the l1 predictions for training meta classifier

In [ ]:
extra_features = ['AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3',
                  'DAYS_EMPLOYED_PERC', 'INCOME_CREDIT_PERC', 'INCOME_PER_PERSON', 'ANNUITY_INCOME_PERC', 
                  'PAYMENT_RATE', 'EXT_SOURCES_MEAN', 'EXT_SOURCES_PRODUCT', 'NEW_PHONE_TO_BIRTH_RATIO']

In [ ]:
extra_features = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 
                 'INCOME_CREDIT_PERC', 'INCOME_PER_PERSON', 'ANNUITY_INCOME_PERC', 'PAYMENT_RATE', 'DAYS_BIRTH', 'DAYS_EMPLOYED']

In [ ]:
extra_features = []

In [ ]:
# Scaler for meta featuers
from sklearn.preprocessing import StandardScaler

def create_meta_dataset(learners, extra_features, scaling=False, add_mean_as_feature=False):
  
  X_train_meta = pd.DataFrame()
  X_test_meta = pd.DataFrame()
  
  for learner in learners:
    clf_name = learner.unique_name
    # each column in train_meta is the pred_proba of a L1-Learner
    X_train_meta[clf_name] = learner.predictions_proba[:, 0]
    
    # test_meta is the mean of the n-fold predictions on test dataset (stil unseen)
    X_test_meta[clf_name] = learner.predictions_proba_test.mean(1)
  
  if add_mean_as_feature:
    X_train_meta['Clf_Mean'] = X_train_meta.mean(axis=1)
    X_test_meta['Clf_mean'] = X_test_meta.mean(axis=1)
    
  # Add extra features
  if len(extra_features) > 0:
    for col in extra_features:
      # get index of the column
      idx = np.where(X_train.columns.values == col)[0][0]
      X_train_meta[col] = X_train.iloc[:,idx]
      idx = np.where(X_test.columns.values == col)[0][0]
      X_test_meta[col] = X_test.iloc[:,idx]
      
  # Scale data
  if scaling:
    scaler_meta = StandardScaler()
    scaler_meta.fit(X_train_meta)
    X_train_meta = pd.DataFrame(scaler_meta.transform(X_train_meta), columns=X_train_meta.columns)
    X_test_meta = pd.DataFrame(scaler_meta.transform(X_test_meta), columns=X_test_meta.columns)
    
  return X_train_meta, X_test_meta

In [ ]:
X_train_meta, X_test_meta = create_meta_dataset(l1_learners, extra_features, scaling=True, add_mean_as_feature=False)
print(X_train_meta.shape)
print(X_test_meta.shape)

# Train meta classifier 

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def print_cv_scores(grid_cv):
  print("Best Score: {}".format(grid_cv.best_score_))
  print("Best Params: {}".format(grid_cv.best_params_))
  for mean, std, params in zip(grid_cv.cv_results_['mean_test_score'], grid_cv.cv_results_['std_test_score'], grid_cv.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

### SGDClassifier

In [ ]:
from sklearn.linear_model import SGDClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [ ]:
clf_meta_sgd = SGDClassifier()

clf_meta_grid_sgd = {
  "loss": Categorical(['log','modified_huber']),
  "penalty": Categorical(['l1', 'l2', 'elasticnet']),
  "l1_ratio": Real(0.00000001, 1, prior='log-uniform'),
  "alpha": Real(0.001, 10, prior='log-uniform'),
  'tol': Real(0.00001, 3, prior='log-uniform'),
  #"class_weight": Categorical([None]),
  #'learning_rate': Categorical(['optimal']),
  #'eta0': Integer(0.1, 1),
#  'max_iter': Integer(5, 500)
}

In [ ]:
gridcv_meta_sgd = BayesSearchCV(clf_meta_sgd, search_spaces=clf_meta_grid_sgd, cv=5, n_jobs=-1, 
                                verbose=3, scoring='roc_auc', n_iter=70, random_state=42)
gridcv_meta_sgd.fit(X_train_meta, y_train)

In [ ]:
print_cv_scores(gridcv_meta_sgd)

In [ ]:
joblib.dump(gridcv_meta_sgd, '{}/{}{}.pkl'.format(stacking_models_dir, 'meta_', 'sgd_with_domainfeatures'))

### LogisticRegression 

In [ ]:
from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [ ]:
clf_meta_logreg = LogisticRegression()
clf_meta_grid_logreg = [
  {
    'C': Real(0.00001, 100, prior='log-uniform'),
    'tol': Real(0.00004, 1, prior='log-uniform'),
    'penalty': Categorical(['l1', 'l2']),
    'random_state': [42],
  },
]

In [ ]:
gridcv_meta_logreg = BayesSearchCV(clf_meta_logreg, search_spaces=clf_meta_grid_logreg, 
                                   cv=5, n_jobs=-1, verbose=3, scoring='roc_auc', n_iter=50)
gridcv_meta_logreg.fit(X_train_meta, y_train)

In [ ]:
print_cv_scores(gridcv_meta_logreg)

In [ ]:
joblib.dump(gridcv_meta_logreg, '{}/{}{}.pkl'.format(stacking_models_dir, 'meta_', 'logreg_with_domainfeatures'))

### SVC 

In [ ]:
from sklearn.svm import SVC

In [ ]:
clf_meta_svc = SVC()
clf_meta_grid_svc = [
  {
    'C': [1], #, 0.1, 1],
    'kernel': ['linear'],
#    'gamma': [0.01, 1.0],
    'class_weight': ['balanced'],
    'random_state': [42],
  },
]

In [ ]:
gridcv_meta_svc = GridSearchCV(clf_meta_svc, clf_meta_grid_svc, cv=5, n_jobs=5, verbose=3, scoring='roc_auc')
gridcv_meta_svc.fit(X_train_meta, y_train)

In [ ]:
print_cv_scores(gridcv_meta_svc)

### XGBoost

In [ ]:
from xgboost import XGBClassifier
clf_meta_xgb = XGBClassifier()

#clf_meta_grid_xgb = {
#  "n_estimators": [25, 100, 250, 300],
#  "max_depth": [4, 8 , 16],
#  "num_leaves": [15, 30]
#  "min_child_weight": [4, 8],
#  "subsample": [0.2, 0.8],
#  "learning_rate": [0.01],
#  "colsample_bytree": [0.8, 0.6],
#  "objective": ['binary:logistic'],
#  "random_state": [42]
#}

# best params
clf_meta_grid_xgb = {
  "n_estimators": [200, 300],
  "max_depth": [4],
  "min_child_weight": [8],
  "subsample": [0.2],
  "learning_rate": [0.01],
  "colsample_bytree": [0.8],
  "objective": ['binary:logistic'],
  "random_state": [42]
}

In [ ]:
gridcv_meta_gdb = GridSearchCV(clf_meta_xgb, param_grid=clf_meta_grid_xgb, cv=5, n_jobs=-1, verbose=5, scoring='roc_auc')
gridcv_meta_gdb.fit(X_train_meta, y_train)

In [ ]:
print_cv_scores(gridcv_meta_gdb)

### XGBoost with Bayesian Optimization

In [ ]:
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from xgboost import XGBClassifier

clf_meta_xgb = XGBClassifier(objective='binary:logistic', eval_metric='auc')

opt = BayesSearchCV(
  clf_meta_xgb, 
  search_spaces=
  {
    "n_estimators": Integer(50, 250),
    "max_depth": Integer(1, 35),
    "min_child_weight": Integer(1, 100),
    "subsample": Real(0.1, 0.9, prior='log-uniform'),
    "learning_rate": Real(0.001, 0.1, prior='log-uniform'),
    "colsample_bytree": Real(0.1, 0.9, prior='log-uniform')
  },
  n_iter=15,
  random_state=42,
  verbose=3,
  scoring = 'roc_auc',
  cv=5,
  n_jobs=-1
)

In [ ]:
opt.fit(X_train_meta, y_train)

In [ ]:
print_cv_scores(opt)

### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
clf_meta_lgm = LGBMClassifier()

clf_meta_grid_lgm = {
  'n_estimators': [250, 280, 290, 300],
  'learning_rate': [0.02, 0.001],
  'num_leaves': [25, 30],
  'max_depth': [16, 20, 25],
  'random_state': [42]
}

In [ ]:
gridcv_meta_lgm = GridSearchCV(clf_meta_lgm, param_grid=clf_meta_grid_lgm, cv=5, n_jobs=-1, verbose=5, scoring='roc_auc')
gridcv_meta_lgm.fit(X_train_meta, y_pred_base)

In [ ]:
print_cv_scores(gridcv_meta_lgm)

### MultiLayer-Perceptron

In [ ]:
from sklearn.neural_network import MLPClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical

In [ ]:
clf_meta_mlp = MLPClassifier()

clf_meta_grid_mlp = {
  #'hidden_layer_sizes': Categorical([(8,5)]),
  'hidden_layer_sizes': Categorical([(3,), (5,), (10,), (8,2)]),
  'activation': Categorical(['relu', 'logistic', 'tanh']),
  'epsilon': Real(1e-9, 1e-4, prior='log-uniform'),
  'alpha': Real(0.000000001, 2, prior='log-uniform'),
}

In [ ]:
gridcv_meta_mlp = BayesSearchCV(clf_meta_mlp, search_spaces=clf_meta_grid_mlp, cv=5, n_jobs=-1, 
                                verbose=5, random_state=49, scoring='roc_auc', n_iter=70)
gridcv_meta_mlp.fit(X_train_meta, y_train)

In [ ]:
print_cv_scores(gridcv_meta_mlp)

In [ ]:
joblib.dump(gridcv_meta_mlp, '{}/{}{}.pkl'.format(stacking_models_dir, 'meta_', 'mlp_without_domainfeatures'))

### Average Predictions of L1-Learners

In [ ]:
y_test = X_test_meta.mean(axis=1)

# Make prediciton for train set and submit csv

In [ ]:
#meta_clf = gridcv_meta_logreg.best_estimator_
#meta_clf = gridcv_meta_gdb.best_estimator_
#meta_clf = gridcv_meta_lgm.best_estimator_
#meta_clf = opt.best_estimator_
#meta_clf = gridcv_meta_sgd.best_estimator_
meta_clf = gridcv_meta_mlp.best_estimator_

y_test = meta_clf.predict_proba(X_test_meta)[:, 1]

In [ ]:
#meta_clf = gridcv_meta_sgd___.best_estimator_
#meta_clf = gridcv_meta_sgd_df.best_estimator_

#meta_clf = gridcv_meta_logreg___.best_estimator_
#meta_clf = gridcv_meta_logreg_df.best_estimator_

#meta_clf = gridcv_meta_mlp___.best_estimator_
meta_clf = gridcv_meta_mlp_df.best_estimator_

y_test = meta_clf.predict_proba(X_test_meta)[:, 1]

In [ ]:
# Linear stretch of predictions to [0,1]
y_test = (y_test - y_test.min()) / (y_test.max() - y_test.min())

In [ ]:
# Make the submission dataframe
submission = pd.DataFrame({'SK_ID_CURR': X_test_SK_IDS, 'TARGET': y_test})
submission = submission.sort_values(by='SK_ID_CURR')

In [ ]:
submission.to_csv('./stacking_submissions/meta_paper/meta_mlp_2_df.csv', index = False)

## Plots

In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import roc_curve, auc

In [ ]:
# Load Grids 
gridcv_meta_sgd___ = joblib.load("{}/{}.pkl".format(stacking_models_dir, 'meta_sgd_without_domainfeatures'))
gridcv_meta_sgd_df = joblib.load("{}/{}.pkl".format(stacking_models_dir, 'meta_sgd_with_domainfeatures'))

gridcv_meta_logreg___ = joblib.load("{}/{}.pkl".format(stacking_models_dir, 'meta_logreg_without_domainfeatures'))
gridcv_meta_logreg_df = joblib.load("{}/{}.pkl".format(stacking_models_dir, 'meta_logreg_with_domainfeatures'))

gridcv_meta_mlp___ = joblib.load("{}/{}.pkl".format(stacking_models_dir, 'meta_mlp_without_domainfeatures'))
gridcv_meta_mlp_df = joblib.load("{}/{}.pkl".format(stacking_models_dir, 'meta_mlp_with_domainfeatures'))

In [ ]:
# SGD (no domain features)
sgd_scores_no_domain = cross_val_predict(gridcv_meta_sgd___.best_estimator_, X_train_meta, y_train, cv=5, n_jobs=-1, method='predict_proba')
#sgd_scores_with_domain = cross_val_predict(gridcv_meta_sgd_df.best_estimator_, X_train_meta, y_train, cv=5, n_jobs=-1, method='predict_proba')

logreg_scores_no_domain = cross_val_predict(gridcv_meta_logreg___.best_estimator_, X_train_meta, y_train, cv=5, n_jobs=-1, method='predict_proba')
#logreg_scores_with_domain = cross_val_predict(gridcv_meta_logreg_df.best_estimator_, X_train_meta, y_train, cv=5, n_jobs=-1, method='predict_proba')

mlp_scores_no_domain = cross_val_predict(gridcv_meta_mlp___.best_estimator_, X_train_meta, y_train, cv=5, n_jobs=-1, method='predict_proba')
#mlp_scores_with_domain = cross_val_predict(gridcv_meta_mlp_df.best_estimator_, X_train_meta, y_train, cv=5, n_jobs=-1, method='predict_proba')

In [ ]:
plt.title('ROC Meta-Learners with Stacking (Method B)')

# SGD (no domain)
fpr, tpr, threshold = roc_curve(y_train, sgd_scores_no_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='SGD (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.76003, 0.75919))

fpr, tpr, threshold = roc_curve(y_train, sgd_scores_with_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='SGD with EF (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75966, 0.75941))

fpr, tpr, threshold = roc_curve(y_train, logreg_scores_no_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='LogReg (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75677, 0.75612))

fpr, tpr, threshold = roc_curve(y_train, logreg_scores_with_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='LogReg with EF (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75947, 0.75856))

fpr, tpr, threshold = roc_curve(y_train, mlp_scores_no_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='NN (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75891, 0.75935))

fpr, tpr, threshold = roc_curve(y_train, mlp_scores_with_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='NN with EF (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.76090, 0.76064))

plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
# SGD (no domain)
fpr, tpr, threshold = roc_curve(y_train, sgd_scores_no_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='SGD (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.76003, 0.75919))

fpr, tpr, threshold = roc_curve(y_train, sgd_scores_with_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='SGD with EF (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75966, 0.75941))

fpr, tpr, threshold = roc_curve(y_train, logreg_scores_no_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='LogReg (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75677, 0.75612))

fpr, tpr, threshold = roc_curve(y_train, logreg_scores_with_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='LogReg with EF (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75947, 0.75856))

fpr, tpr, threshold = roc_curve(y_train, mlp_scores_no_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='MLP (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.75891, 0.75935))

fpr, tpr, threshold = roc_curve(y_train, mlp_scores_with_domain[:,1])
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label='MLP with EF (CV=%0.5f, PubLB=%0.5f, PriLB=%0.5f)' % (roc_auc, 0.76090, 0.76064))

plt.grid()
plt.title('ROC Meta-Learners with Stacking (Method B)', fontsize=14)
plt.xlabel('False positive rate', fontsize=14)
plt.ylabel('True positive rate', fontsize=14)
plt.legend(loc="lower right", prop={'size': 14})
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.show()

In [ ]:
sns.reset_orig()